In [1]:
import os 
import pickle 
import torch 

with open(os.path.expanduser('~/dataset/OAG/OAG-CS/hg_full.dict.pkl'), 'rb') as fp: 
    graph_info_dict = pickle.load(fp) 
    
PV_edge_index = graph_info_dict['edge_index_dict'][('paper', 'PV_Journal', 'venue')]
paper_year = graph_info_dict['paper_year'] 
num_paper_nodes = len(paper_year) 

PV_edge_index.shape, paper_year.shape 

(torch.Size([2, 228062]), torch.Size([544244]))

In [2]:
PV_adj_dict: dict[int, int] = dict() 

for P_nid, V_nid in zip(*PV_edge_index.tolist()): 
    PV_adj_dict[P_nid] = V_nid 
    
paper_venue_label = torch.full(fill_value=-1, size=[num_paper_nodes], dtype=torch.int64) 
paper_train_mask = torch.zeros(num_paper_nodes, dtype=torch.bool)
paper_val_mask = torch.zeros(num_paper_nodes, dtype=torch.bool)
paper_test_mask = torch.zeros(num_paper_nodes, dtype=torch.bool)

venue_remap: dict[int, int] = dict() 

for P_nid, P_year in enumerate(paper_year.tolist()): 
    if P_year >= 2014: 
        if P_nid in PV_adj_dict: 
            V_nid = PV_adj_dict[P_nid] 
            
            if V_nid not in venue_remap: 
                venue_remap[V_nid] = len(venue_remap) 
                
            label_id = venue_remap[V_nid]
            
            paper_venue_label[P_nid] = label_id 
            
            if P_year <= 2016:
                paper_train_mask[P_nid] = True 
            elif P_year == 2017:
                paper_val_mask[P_nid] = True 
            else:
                paper_test_mask[P_nid] = True 

int(torch.max(paper_venue_label)) + 1, \
    paper_train_mask.sum(), \
    paper_val_mask.sum(), \
    paper_test_mask.sum(), \
    paper_train_mask.float().mean(), \
    paper_val_mask.float().mean(), \
    paper_test_mask.float().mean()

(2442,
 tensor(48799),
 tensor(19494),
 tensor(27217),
 tensor(0.0897),
 tensor(0.0358),
 tensor(0.0500))

In [3]:
paper_pretrain_mask = ~(paper_train_mask | paper_val_mask | paper_test_mask)

paper_pretrain_mask.sum(), \
    paper_pretrain_mask.float().mean() 

(tensor(448734), tensor(0.8245))

In [4]:
graph_info_dict['paper_venue_label'] = paper_venue_label 
graph_info_dict['paper_pretrain_mask'] = paper_pretrain_mask 
graph_info_dict['paper_train_mask'] = paper_train_mask 
graph_info_dict['paper_val_mask'] = paper_val_mask 
graph_info_dict['paper_test_mask'] = paper_test_mask 

with open(os.path.expanduser('~/dataset/OAG/OAG-CS/hg_venue.pkl'), 'wb') as fp: 
    pickle.dump(graph_info_dict, fp) 